# Imports

In [ ]:
import os
import time
import tensorflow as tf
import numpy as np
from glob import glob
import datetime
import random
from PIL import Image
import matplotlib.pyplot as plt

# Load in Data as Numpy Array

In [ ]:
DATA_PATH = "./data/"
BATCH_SIZE = 64
DATA_NAMES = [DATA_PATH + str(line).rstrip() for line in open("./100k.txt","r")]

In [1]:
def get_batch(data_names: list, batch_size: int):
    """
        Grab a certain number of random images from the dataset
        Parameters:
            data_names: the list of names of images
            batch_size: how many images to grab 
    """
    files = random.sample(data_names, batch_size)
    batch = []
    for file in files:
        if random.choice([True, False]):
            batch.append(np.asarray(Image.open(file).transpose(Image.FLIP_LEFT_RIGHT)))
        else:
            batch.append(np.asarray(Image.open(file)))                     
    batch = np.asarray(batch)
    normalized_batch = (batch / 127.5) - 1.0 # As you'd usually do
    return normalized_batch, files

# Discriminator

In [ ]:
def discriminator(input_z):
    """
        logits is used to calculate loss
        output is simply needed for the shape (which should theoretically be deriveable from from logits but that's beside the point)
    """
    logits = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(3, 7, 1, activation='relu', input_shape=(256, 256, 3)),
        tf.keras.layers.Conv2D(32, 5, (1, 2), activation='relu'),
        tf.keras.layers.Conv2D(64, 5, 2, activation='relu'),
        tf.keras.layers.Conv2D(128, 5, 2, activation='relu'),
        tf.keras.layers.Conv2D(256, 3, 2, activation='relu'),
        tf.keras.layers.Conv2D(512, 3, 2, activation='relu'),
        tf.keras.layers.Conv2D(512, 3, 1, activation='relu'),
        tf.keras.layers.MaxPool2D(2, 2),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1, activation=None),
    ])
    output = tf.sigmoid(logits)
    # Notice how we don't really compile the model here, this is because we'll have to implement our own custom cost function
    return output, logits

# Generator


In [ ]:
def generator(input_z, output_channel_dim, training):
    """
        Obviously, the deepfake generator
        
    """